In [60]:
# Generated code -- http://www.redblobgames.com/grids/hexagons/
import collections
import math, random
import itertools
import os, copy
from copy import deepcopy
import time
import pandas as pd

def initializeBoard():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.randint(0,1)
            z = -x-y
            Hex[(x, y, z)] = ''#side[rnd]
    return Hex

def allWhiteBoardZobrist():
    whitezobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            whitezobristboard[(x, y, z)] = rnd
    return whitezobristboard

def allBlackBoardZobrist():
    blackzobristboard = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            blackzobristboard[(x, y, z)] = rnd
    return blackzobristboard

def initializeBoardZobrist():
    Hex = {}

    for x in range(-N, N+1):
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):
            rnd = random.getrandbits(64)
            z = -x-y
            Hex[(x, y, z)] = rnd
    return Hex

def boardToZobrist(board, zboard, wzobristboard, bzobristboard):
    random.seed(1356)
    zobristkey = random.getrandbits(64)
    wzobrists = wzobristboard.values()
    bzobrists = bzobristboard.values()
    
    for cel in board:
        if board[cel] == 'b':
            zobristkey ^= (bzobristboard[cel] ^ zboard[cel])
        elif board[cel] == 'w':
            zobristkey ^= (wzobristboard[cel] ^ zboard[cel])
        
    return zobristkey

# Form groups and calculate nr of members
def EvalBoard(board):
    score = find_scores(board, 'b') #score of player1
    score2 = find_scores(board, 'w') #score of player2

    group = findGroups(score)
    group2 = findGroups(score2)

    player_one_total, player_two_total = result(group, group2)
    tempscore = 0
  
    return player_one_total, player_two_total, player_one_total - player_two_total

def find_scores(board, side):   
    tot = {}
    
    listOfkeys = [key for (key, value) in board.items() if value == side]
    
    for lok  in listOfkeys:
        tot[lok] = side
        
    return tot

def listOfEmptySpots(board):
    return [k for k, v in board.items() if v == '']

def neighbours(tiles, scoregroup, visited):
    group = []
    for tile in tiles:
        for direction in hex_directions:
            new_tile2 = tuple(map(sum, zip(tile, direction)))
            if new_tile2 in scoregroup and new_tile2 not in visited:
                group += [new_tile2]
                visited += [new_tile2]
    return group, visited

def findGroups(score):
    newlist = []
    visited = []
    for sc in score:
        temp = [sc]
        if sc not in visited:
            visited += [sc]
            groupt, visited = neighbours([sc], score, visited)
            temp += groupt
            visited += groupt
            find = True
            while find:
                groupt, visited = neighbours(groupt, score, visited)
                if len(groupt) != 0:
                    temp += groupt
                else:
                    find = False
                    
            newlist += [len(temp)]
    return newlist


# multiply all group counts of the same color for final game score
from functools import reduce

def multiplyall(count_list):
    if count_list != []:
        return reduce(lambda x, y: x*y, count_list)
    return 0

def result(count_list, count_list2):
    game_total = multiplyall(count_list)
    game_total2 = multiplyall(count_list2)
    return game_total, game_total2

# Put pieces on board
def makeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

# Put pieces on board
def unmakeMove(board, move, color):
    x, y = move
    board[(x, y, -x-y)] = color
    return board

def isEmpty(board, position):
    if position in board:
        return len(board[position]) == 0
    else:
        return False

def fullBoard(board):
    number_ofcells = len(board)
    number_ofstones = 0
    for cell in board:
        if board[cell] != '':
            number_ofstones += 1
    return number_ofstones >= number_ofcells -1


#Check if a move is valid
def validMove(board, x, y):
    
    #Check if there is a piece on the location
    if not isEmpty(board, (x, y, -x-y)):
        return False
    
    #Check if the piece is on the board
    if abs(x) > N or abs(y) > N:
        return False
    
    return True

def moves_left(board):
    print('These are the moves left: ')
    
    moves = ''
    counter = 0
    for pos in board:
        
        if board[pos] == '':
            moves += str(pos)
            counter += 1
            if counter == 7:
                moves += '\n'
                counter = 0
            else:
                moves += ' '
    print (moves)
    
def printBoard(board):
    for x in range(-N, N+1):
        string = ''
        if x > 0:
            string += ' ' * (N-(N-x))
        else:
            string += ' ' * -(N-(N-x))
        for y in range(max(-N, -x-N) , min(+N, -x+N)+1):

            if (x, y, -x-y) in board:
                if board[(y, -x-y, x)] == '':
                    string += '_ '
                else:
                    string += board[(y, -x-y, x)] + ' '
        print(string)     
        
def zobristStateScore(board):
    # Create a zobrist key for the new board/childnode
    zidentity = boardToZobrist(board, zboard, wzobristboard, bzobristboard)
    maxplayer, minplayer, tempscore = EvalBoard(board)
        
    return zidentity%1000, tempscore, maxplayer, minplayer


# if no valid moves possible then True
def leafNode(board):
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    
    bestValue = -INFINITY
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]
        #Make two valid moves
        if validMove(board, x, y) and validMove(board, x2, y2):
            return False
    return True

In [20]:
random.seed(1356) # set seed so that each play is the same

hex_directions = [(1, 0, -1), (1, -1, 0), (0, -1, 1), (-1, 0, 1), (-1, 1, 0), (0, 1, -1)]
hex_diagonals = [(2, -1, -1), (1, -2, 1), (-1, -1, 2), (-2, 1, 1), (-1, 2, -1), (1, 1, -2)]

side = [random.getrandbits(64), random.getrandbits(64)]#[b, w, empty] # side to move ['b', 'w']#

N = 4 # Number of hexes side
p = 2 # number of players

blackstones = 136
whitestones = 136

board = initializeBoard() # board with assigned black and white pieces
zboard = initializeBoardZobrist() # initial board, without any pieces

wzobristboard = allWhiteBoardZobrist() # all hexes filled with black pieces
bzobristboard = allBlackBoardZobrist() # all hexes filled with white pieces

INFINITY = 999

In [31]:
import itertools
from copy import deepcopy

def NegamaxAB(board, depth, alpha, beta, side):
    if leafNode(board) or len(listOfEmptySpots(board)) < 2 or depth == 0:
        return -side * EvalBoard(board)[2]

    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_w = deepcopy(board) # make sure that you copy the board from the parent!!
    
    bestValue = -INFINITY
    for cell in all_cells:
        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board_w, x, y) and validMove(board_w, x2, y2):
            makeMove(board_w, [x, y], 'b')
            makeMove(board_w, [x2, y2], 'w')
            
            boardZobrist = boardToZobrist(board_w, zboard, wzobristboard, bzobristboard)
            index = boardZobrist % 1000 #% len(tt)

            if index in tt:
                continue

            index, points, _, minplayer = zobristStateScore(board_w)

            tt[index] = [depth, state, points]
        
            v = -NegamaxAB(board_w, depth-1, -beta, -alpha, -side)
            bestValue = max(bestValue, v)
            
            alpha = max(alpha, v)
            if alpha >= beta:
#                 print('AB', alpha, beta, alpha >= beta, bestValue)
                all_cells = all_cells[:all_cells.index(cell)]
                break
    
    return bestValue

In [33]:
def BestMove(board):
    maxPoints = 0
    mx =  N+1; 
    my =  N+1
    mx2 = N+1; 
    my2 = N+1
    
    all_cells = list(itertools.permutations(list(board), 2)) # find all cell of the board and create move pairs.
    board_temp = deepcopy(board) # make sure that you copy the board from the parent!!
    
    for cell in all_cells:

        x, y, _ = cell[0]
        x2, y2, _ = cell[1]

        #Make two valid moves
        if validMove(board_temp, x, y) and validMove(board_temp, x2, y2):
            makeMove(board_temp, [x, y], 'b')
            makeMove(board_temp, [x2, y2], 'w')

            boardZobrist = boardToZobrist(board, zboard, wzobristboard, bzobristboard)
            index = boardZobrist % 1000
            if index in tt:
                points = tt[index][-1]

                if points >= maxPoints:
                    maxPoints = points
                    mx = x; my = y
                    mx2 = x2; my2 = y2

                    continue
            points = NegamaxAB(board_temp, depth, -INFINITY, +INFINITY, 1)

#             print(points, maxPoints)
            if points >= maxPoints:
                maxPoints = points
                mx = x; my = y
                mx2 = x2; my2 = y2
    
    
    return [mx, my], [mx2, my2], board_temp

In [65]:
# def startGame(play):
tt = {}
gamehistory = {}
points = 0
   
opt = int(input('Omega game initialised\n\nSelect play option: \n0: Man Vs Machine\n1: Machine Vs Man\n2: AI vs AI\n\n'))

depth = int(input('Select search depth:\n'))
play = True
players = ['White', 'Black', '', 'AI']
player = 0
played = 0
counter = 0

if opt == 0:
    print ('Selected option 1\n', '\n1: User (w)\n2: AI (b)\n(Press Enter for Exit!)')
    player = opt
elif opt == 1:
    print ('Selected option 1\n', '\n1: AI (w)\n2: User (b)\n(Press Enter for Exit!)')
    player = opt
else:
    print ('Selected option default\n', '\n1: AI (w)\n2: AI (b)\n(Press Enter for Exit!)')
    player = 3
    
board = initializeBoard()


def menuOptions(xy):
    if xy == '':
        os._exit(0)
        
    if xy == '+': 
        unmakeMove(board, [int(x), int(y)], 'b')
        unmakeMove(board, [int(x2), int(y2)], 'w')
        print('Undone', [int(x), int(y)], [int(x2), int(y2)])
        xy = input('X,Y: ')

#     Check available moves
    if xy == '?': 
        moves_left(board)
        xy = input('X,Y: ')
    

while play:
    if counter == 0:
        printBoard(board)
    
    if fullBoard(board):
        print ('No moves left! Game ended!')
        print ('Score User: ' + str(EvalBoard(board) [0]))
        print ('Score AI  : ' + str(EvalBoard(board) [1]))
        if EvalBoard(board) [0] > EvalBoard(board) [1]:
            
            print('Player b wins')
        elif EvalBoard(board) [0] == EvalBoard(board) [1]:
            print('It\'s a tie!')  
        else:
            print('AI wins')

    
    if player == 0:
        if counter == 0:
            print ('The next move is for', players[player], 'player.\n')
        counter += 1
        
        if played == 0:
            xy = input('X,Y: ')
            menuOptions(xy) # check for chosen options

    #         if played == 0:
            (x, y) = xy.split(',')
            x = int(x)
            y = int(y)

            if validMove(board, x, y):
                makeMove(board, [x, y], 'w')
                print('w', (int(x), int(y), -int(x)-int(y)))

                played = 1
            else:
                print ('Invalid move! Try again!')
                continue


    if played == 1:
        xy = input('X,Y: ')
        menuOptions(xy) # check for chosen options

        (x, y) = xy.split(',')
        x2 = int(x)
        y2 = int(y)

        if validMove(board, x2, y2):
            makeMove(board, [x2, y2], 'b')
            print('b', (int(x2), int(y2), -int(x2)-int(y2)))

            state = ''
            if EvalBoard(board) [0] > EvalBoard(board) [1]:
                state = 'Win'
            elif EvalBoard(board) [0] == EvalBoard(board) [1]:
                state = 'Draw' 
            else:
                state = 'Lose'

            played = 0
            player = 1
            move_made = 1
            counter = 0
            continue
            
        else:
            print ('Invalid move! Try again2!')
            continue
            
    else: # AI 
        if counter == 0:
            print ('The next move is for', players[player], 'player.\n')
            
        counter += 1
        start = time.time()
        moveb, movew, boardTemp = BestMove(board)
        if counter == 0:
            print('Thinking time: ', time.time() - start, moveb, movew)
        
        if moveb[0] != N +1:
            makeMove(board, moveb, 'w')
            makeMove(board, movew, 'b')
            
            state = ''
            if EvalBoard(boardTemp) [0] > EvalBoard(boardTemp) [1]:
                state = 'Win'
            elif EvalBoard(boardTemp) [0] == EvalBoard(boardTemp) [1]:
                state = 'Draw' 
            else:
                state = 'Lose'
                
            if player == 1:
                player = 0
            counter = 0
        
            

Omega game initialised

Select play option: 
0: Man Vs Machine
1: Machine Vs Man
2: AI vs AI

2
Select search depth:
3
Selected option default
 
1: AI (w)
2: AI (b)
(Press Enter for Exit!)
    _ _ _ _ _ 
   _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 
The next move is for AI player.

    _ _ _ _ _ 
   _ _ _ _ _ b 
  _ _ _ _ _ _ w 
 _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   _ _ _ _ _ _ 
    _ _ _ _ _ 
The next move is for AI player.

    _ _ _ _ _ 
   _ _ _ _ _ b 
  _ _ _ _ _ _ w 
 _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   b _ _ _ _ _ 
    w _ _ _ _ 
The next move is for AI player.

    _ _ _ _ _ 
   _ _ _ _ _ b 
  _ _ _ _ _ _ w 
 _ _ _ _ _ _ _ b 
_ _ _ _ _ _ _ _ w 
 _ _ _ _ _ _ _ _ 
  _ _ _ _ _ _ _ 
   b _ _ _ _ _ 
    w _ _ _ _ 
The next move is for AI player.

    _ _ _ b _ 
   _ _ _ _ w b 
  _ _ _ _ _ _ w 
 _ _ _ _ _ _ _ b 
_ _ _ _ _

KeyboardInterrupt: 

In [ ]:
# function negamax(node, depth, α, β, color) is
#     alphaOrig := α

#     (* Transposition Table Lookup; node is the lookup key for ttEntry *)
#     ttEntry := transpositionTableLookup(node)
#     if ttEntry is valid and ttEntry.depth ≥ depth then
#         if ttEntry.flag = EXACT then
#             return ttEntry.value
#         else if ttEntry.flag = LOWERBOUND then
#             α := max(α, ttEntry.value)
#         else if ttEntry.flag = UPPERBOUND then
#             β := min(β, ttEntry.value)

#         if α ≥ β then
#             return ttEntry.value

#     if depth = 0 or node is a terminal node then
#         return color × the heuristic value of node

#     childNodes := generateMoves(node)
#     childNodes := orderMoves(childNodes)
#     value := −∞
#     for each child in childNodes do
#         value := max(value, −negamax(child, depth − 1, −β, −α, −color))
#         α := max(α, value)
#         if α ≥ β then
#             break

#     (* Transposition Table Store; node is the lookup key for ttEntry *)
#     ttEntry.value := value
#     if value ≤ alphaOrig then
#         ttEntry.flag := UPPERBOUND
#     else if value ≥ β then
#         ttEntry.flag := LOWERBOUND
#     else
#         ttEntry.flag := EXACT
#     ttEntry.depth := depth	
#     transpositionTableStore(node, ttEntry)

#     return value

In [67]:
board[(0, 4, -4)] = 'w'

In [68]:
cells = list(board)

counter = len(cells)-1
for x in range(-N, N+1):
    string = ''
    if x > 0:
        string += ' ' * (N-(N-x))
    else:
        string += ' ' * -(N-(N-x))
    for y in range(max(-N, -x-N) , min(+N, -x+N)+1):

        if (x, y, -x-y) in board:
            if board[(y, -x-y, x)] == '':
                string += '_ '
            else:
                string += board[(y, -x-y, x)] + ' '
        counter -= 1
    print(string)

    w _ b b _ 
   _ _ _ w w b 
  _ _ _ _ b b w 
 _ _ _ _ _ w w b 
_ _ _ _ _ _ _ b w 
 _ _ _ _ _ _ _ w 
  _ _ _ _ _ _ _ 
   b _ _ _ _ _ 
    w _ _ _ _ 


In [51]:
len(cells)

61

In [ ]:
# def GetSortedNodes(board, player):
#     sortedNodes = []
#     for y in range(n):
#         for x in range(n):
#             if ValidMove(board, x, y, player):
#                 (boardTemp, totctr) = MakeMove(copy.deepcopy(board), x, y, player)
#                 sortedNodes.append((boardTemp, EvalBoard(boardTemp, player)))
#     sortedNodes = sorted(sortedNodes, key = lambda node: node[1], reverse = True)
#     sortedNodes = [node[0] for node in sortedNodes]
#     return sortedNodes

In [ ]:
# """
# The standard AI algorithm of easyAI is Negamax with alpha-beta pruning
# and (optionnally), transposition tables.
# """
 
# import pickle
         
# LOWERBOUND, EXACT, UPPERBOUND = -1,0,1
# inf = float('infinity')
 
# def negamax(game, depth, origDepth, scoring, alpha=+inf, beta=-inf,
#              tt=None):
#     """
#     This implements Negamax with transposition tables.
#     This method is not meant to be used directly. See ``easyAI.Negamax``
#     for an example of practical use.
#     This function is implemented (almost) acccording to
#     http://en.wikipedia.org/wiki/Negamax
#     """
     
#     alphaOrig = alpha
     
#     # Is there a transposition table and is this game in it ?
#     lookup = None if (tt is None) else tt.lookup(game)
     
#     if lookup != None:
#         # The game has been visited in the past
         
#         if lookup['depth'] >= depth:
#             flag, value = lookup['flag'], lookup['value']
#             if flag == EXACT:
#                 if depth == origDepth:
#                     game.ai_move = lookup['move']
#                 return value
#             elif flag == LOWERBOUND:
#                 alpha = max( alpha, value)
#             elif flag == UPPERBOUND:
#                 beta = min( beta, value)
                 
#             if alpha >= beta:
#                 if depth == origDepth:
#                     game.ai_move = lookup['move']
#                 return value
         
         
         
     
#     if (depth == 0) or game.is_over():
#         score = scoring(game)
#         if score == 0:
#             return score
#         else:
#             return  (score - 0.01*depth*abs(score)/score)
     
     
#     if lookup != None:
#         # Put the supposedly best move first in the list
#         possible_moves = game.possible_moves()
#         possible_moves.remove(lookup['move'])
#         possible_moves = [lookup['move']] + possible_moves
         
#     else:
         
#         possible_moves = game.possible_moves()
     
     
     
#     state = game
#     best_move = possible_moves[0]
#     if depth == origDepth:
#         state.ai_move = possible_moves[0]
         
#     bestValue = -inf
#     unmake_move = hasattr(state, 'unmake_move')
     
     
#     for move in possible_moves:
         
#         if not unmake_move:
#             game = state.copy() # re-initialize move
         
#         game.make_move(move)
#         game.switch_player()
         
#         move_alpha = - negamax(game, depth-1, origDepth, scoring,
#                                -beta, -alpha, tt)
         
#         if unmake_move:
#             game.switch_player()
#             game.unmake_move(move)
         
#         bestValue = max( bestValue,  move_alpha )
#         if  alpha < move_alpha :
#                 alpha = move_alpha
#                 best_move = move
#                 if depth == origDepth:
#                     state.ai_move = move
#                 if (alpha >= beta):
#                     break
         
#     if tt != None:
         
#         assert best_move in possible_moves
#         tt.store(game=state, depth=depth, value = bestValue,
#                  move= best_move,
#                  flag = UPPERBOUND if (bestValue <= alphaOrig) else (
#                         LOWERBOUND if (bestValue >= beta) else EXACT))
 
#     return bestValue